In [2]:
import matplotlib.pyplot as pl
import pyqtgraph as pg
import os, sys, time, datetime,argparse,datetime,imutils,threading, cv2
import numpy as np
from time import sleep
from glob import glob
from imp import reload
from threading import Thread
from MyTimer import MyTimer
from MyArduino import MyArduino
from RawImageWidget import RawImageWidget
from PIL import Image
from MyGif import MyGif
from PyQt5.QtGui import *
import record_video as rv
from MyQtWidget import MyVideoRecorder
os.chdir('/Users/virginiarutten/Documents/deep_fish/')

% matplotlib inline
% gui qt

## video tracking

### record file

In [ ]:
! python /Users/virginiarutten/Documents/deep_fish/tracking/record_video.py -t 10 -n 'test_'

### load file

In [3]:
foldername = '/Users/virginiarutten/Documents/deep_fish/videos/'
dt = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")[2:]

In [27]:
rv = MyVideoRecorder()

connecting to camera 0...
320 180


In [13]:
import serial.tools.list_ports
ports = list(serial.tools.list_ports.comports())

In [23]:
ports
for p in ports:
    if list(p)[1] =='Arduino Uno':
        print(list(p)[0])

/dev/cu.usbmodem1461


In [24]:
list(tmp)[0]

'/dev/cu.usbmodem1461'

In [4]:
save_btn = QCheckBox("save")

In [11]:
save_btn.setChecked(False)


In [12]:
save_btn.isChecked()

False

In [47]:
from pyqtgraph import QAPP

In [48]:
app = QAPP([])

TypeError: 'NoneType' object is not callable

In [49]:
QAPP??

Type:        NoneType
String form: None
Docstring:   <no docstring>


In [40]:
save_btn.isChecked()

False

In [19]:
dt[:20]

'180408-063826'

In [29]:
cv2.VideoWriter??

Docstring:
VideoWriter() -> <VideoWriter object>
.   @brief Default constructors
.   
.   The constructors/functions initialize video writers.
.   -   On Linux FFMPEG is used to write videos;
.   -   On Windows FFMPEG or VFW is used;
.   -   On MacOSX QTKit is used.



VideoWriter(filename, fourcc, fps, frameSize[, isColor]) -> <VideoWriter object>
.   @overload
.   @param filename Name of the output video file.
.   @param fourcc 4-character code of codec used to compress the frames. For example,
.   VideoWriter::fourcc('P','I','M','1') is a MPEG-1 codec, VideoWriter::fourcc('M','J','P','G') is a
.   motion-jpeg codec etc. List of codes can be obtained at [Video Codecs by
.   FOURCC](http://www.fourcc.org/codecs.php) page. FFMPEG backend with MP4 container natively uses
.   other values as fourcc code: see [ObjectType](http://www.mp4ra.org/codecs.html),
.   so you may receive a warning message from OpenCV about fourcc code conversion.
.   @param fps Framerate of the created video strea

In [20]:
f = foldername + dt[:6]
print(f)

/Users/virginiarutten/Documents/deep_fish/videos/180408


In [23]:
os.mkdir(f)

True

In [16]:
f

'/Users/virginiarutten/Documents/deep_fish/videos/180408'

In [8]:
default_path = '/Users/virginiarutten/Documents/deep_fish/videos/180404/HD Webcam C525/'
files = glob(default_path+'/*25*')
default_path = '/Users/virginiarutten/Documents/deep_fish/videos/180405/'
files = glob(default_path+'/*')

print(files)

['/Users/virginiarutten/Documents/deep_fish/videos/180405/2018-04-06 22:34:40.mov', '/Users/virginiarutten/Documents/deep_fish/videos/180405/HD Webcam C525', '/Users/virginiarutten/Documents/deep_fish/videos/180405/2018-04-06 22:34:40.mov.gif', '/Users/virginiarutten/Documents/deep_fish/videos/180405/2018-04-06 22:34:40.mp4', '/Users/virginiarutten/Documents/deep_fish/videos/180405/Record_File.mov', '/Users/virginiarutten/Documents/deep_fish/videos/180405/2018-04-06 22:34:40_1.mov', '/Users/virginiarutten/Documents/deep_fish/videos/180405/2018-04-06 22:34:40_1.gif', '/Users/virginiarutten/Documents/deep_fish/videos/180405/2018-04-06 22:34:40_1 2.gif']


In [9]:
file = files[0]
print(file)

/Users/virginiarutten/Documents/deep_fish/videos/180405/2018-04-06 22:34:40.mov


### functions

In [10]:
def grab_frame(camera, fx=1, opt=0):
    try:
        (grabbed, frame) = camera.read()
        frame = cv2.resize(frame, (0,0), fx= fx,fy = fx)
        imW = frame.shape[1]
        imH = frame.shape[0]
        if opt==1:
            return frame, imW, imH
        else:
            return frame
    except:
        print('couldn\'t grab frame')
        if opt==1:
            return None, None, None
        else:
            return None
        
def normalise(ims: np.ndarray):
    if not isinstance(ims, np.ndarray):
        raise TypeError
    ims_avg = ims.mean(axis = 0)
    frameDelta = ims_avg[None,:,:]-ims
    min_ = frameDelta.min()
    if min_<0:
        frameDelta -=min_
    else:
        frameDelta +=min_
    frameDelta /= frameDelta.max()/254
    return frameDelta, ims_avg


def get_stats(file):
    cap = cv2.VideoCapture(file)
    length = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    width  = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps    = cap.get(cv2.CAP_PROP_FPS)
    cap.release()
    return length, width, height, fps




def mask_frame(fD, lLim, uLim):
    mask = cv2.inRange(fD,  lLim, uLim)
    masked = cv2.bitwise_or(fD, fD, mask=mask)   
    return mask, masked


def mask_frames(ims, lLim=30, uLim = 80):
    mask = []
    masked = []

    for i in range(ims.shape[0]):
        mask_, masked_ = mask_frame(ims[i], lLim, uLim)
        mask.append(mask_)
        masked.append(masked_)
    mask = np.array(mask)
    masked = np.array(masked)
    return mask, masked

def countour_frames(masked,area_mn = 30, area_mx = 2000, fish_num = 4):
    im_ = []
    keypoints = {}
    for i in range(masked.shape[0]):
        m_tmp,keypoints = get_frame_contours(masked[i], keypoints, area_mn,area_mx, fish_num)
        im_.append(m_tmp)

    im_ = np.array(im_)
    return im_, keypoints


def get_frame_contours(masked, keypoints, area_mn, area_mx = 13*37, fish_num = 4):   
    
        _, cnts, _ = cv2.findContours(image = masked, mode = cv2.RETR_EXTERNAL,\
                                      method = cv2.CHAIN_APPROX_SIMPLE)
        
        im_ = np.repeat(masked.copy()[:,:,None], repeats = 3,axis = 2)

        
        x = np.array([])
        y = np.array([])
        cX = np.array([])
        cY = np.array([])
        h = np.array([])
        w = np.array([])
        C = []
        
        a = np.array([])
        for c in cnts:
            a_ = cv2.contourArea(c) 
            # if the contour is too small, ignore it
            if a_ > area_mn:
                if a_ < area_mx*1000:
                    a = np.concatenate([a,np.array([a_])])
                    
                    M = cv2.moments(c)
                    if M["m00"]<1e-9:
                        M["m00"] = 1e10
                    cX_ = int(M["m10"] / M["m00"])
                    cY_ = int(M["m01"] / M["m00"])

                    cX = np.concatenate([cX,np.array([cX_])])
                    cY = np.concatenate([cY,np.array([cY_])]) 

                    C.append(c)
                    (x_, y_, w_, h_) = cv2.boundingRect(c)
                    x = np.concatenate([x,np.array([x_])])
                    y = np.concatenate([y,np.array([y_])])  
                    h = np.concatenate([h,np.array([h_])])
                    w = np.concatenate([w,np.array([w_])])  
                    
            
        ## get indices of largest areas
        if len(a)==0:
            max_a = -1
        else:
            max_a = np.max(a)
        locs = np.argsort(-a[:fish_num])
        x = x[locs].astype('int')
        y = y[locs].astype('int')
        cX = cX[locs].astype('int')
        cY = cY[locs].astype('int')
        h = h[locs].astype('int')
        w = w[locs].astype('int')
        a = a[locs].astype('int')
        C = np.array(C)
        C = C[locs]
        
        
        cXX =cX.reshape([1,-1])
        cYY =cY.reshape([1,-1])
        xx =x.reshape([1,-1])
        yy =y.reshape([1,-1])
        hh =h.reshape([1,-1])
        ww =w.reshape([1,-1])
        

                          
        keylen = len(keypoints)
        keypoints[keylen  + 1] = np.concatenate([cXX, cYY, xx,yy,hh,ww], axis = 0).T
        keylen = len(keypoints)
        
        for i in range(locs.shape[0]):
#             im_ = cv2.rectangle(im_, (x[i], y[i]), (x[i] + int(w[i]), y[i] + int(h[i])), (255, 255, 0), 1)
            im_ = cv2.circle(im_, (cX[i], cY[i]), radius=2, color=(0, 255, 0), thickness=3)
            im_ = cv2.drawContours(im_, C[i], -1, (255,0,255), 1)
            
        pre_fr = 20
        if keylen>25:
            for buf in range(pre_fr):
                locs_ = keypoints[keylen-buf-1]
                for i in range(locs_.shape[0]):
                    im_ = cv2.circle(im_, (locs_[i,0], locs_[i,1]), radius=1, color=(255, 0, 255), thickness=1)

        return im_, keypoints
    
def image(image_ = None,title = 'frame', pos = (0,0)):
    win = pg.ImageView()
    if image_!=None:
        win.setImage(image_)
    win.show()
    win.setWindowTitle(title)
    win.move(pos[0], pos[1])
    return win

def find_trajs(keypoints, max_dist = 20):
    fr_num = len(keypoints)
    # for fr in range(fr_num-1):  

    init_key = keypoints[1][:,0:2]
    pre_count = init_key.shape[0]
    trajs = np.zeros([pre_count,fr_num,2])
    trajs[:,0,:] = init_key

    for fr in range(fr_num-2):      
        locs = keypoints[fr+2][:,0:2]
        new_count = locs.shape[0]    
        active_prelocs = trajs[:,fr,0]>0
        prelocs = trajs[active_prelocs,fr]
        pre_count = prelocs.shape[0]

        if new_count>pre_count:
    #         print('new fish located')
            trajs = np.vstack([trajs, np.zeros([1, trajs.shape[1],trajs.shape[2]])])

        dists = []
        ords = []

        for new in range(new_count):
            dist_ = []
            for old in range(pre_count):
                dist_.append(np.linalg.norm(prelocs[old]-locs[new]))
            dist_ = np.array(dist_)
            dists.append(dist_)
            ord_ = np.argsort(dist_)
            ords.append(ord_)

        ## new by old
        dists = np.array(dists)
        dists_buf = np.copy(dists)
        ords = np.array(ords)
        dists_buf[dists_buf>max_dist] = 1e10
        
        located_fish = np.array([90])
        unlocated_fish = list(set(np.arange(new_count)))

        while len(unlocated_fish)>0:
    #     for i in range(new_count):
            unlocated_fish = list(set(np.arange(new_count))- set(located_fish))
            if pre_count==0:
                min_ = max_dist+1
            else:
                min_ = dists_buf.min()
            if min_>max_dist:  
                if len(unlocated_fish)>0:
                    fish_ind = unlocated_fish[0]
                    trajs = np.vstack([trajs, np.zeros([1, trajs.shape[1],trajs.shape[2]])])
                    trajs[-1, fr+1, :] =locs[fish_ind] 
                    located_fish = np.hstack([fish_ind,located_fish] )
            else:
                best_new_fish = np.where(dists_buf== dists_buf.min())[0]
                best_prev_fish = np.where(dists_buf== dists_buf.min())[1]
                if len(best_new_fish)>1:
                    best_new_fish = best_new_fish[0]
                    best_prev_fish = best_prev_fish[0]

                located_fish = np.hstack([best_new_fish,located_fish])

                best_prv_loc = np.where(active_prelocs==True)[0][best_prev_fish]
                trajs[best_prv_loc,fr+1,:] = locs[best_new_fish]
                dists_buf[:, best_prev_fish]= 1e10
                dists_buf[best_new_fish,:]= 1e10
    return trajs

def delta_frame(ims: np.ndarray, win = 500):
    ims = ims.astype('uint8')
    ttFr = ims.shape[0]
    diffs =[]
    for i in range(ttFr):
        if i<=win/2:
            ims_avg_ = ims[0:int(i+win/2)].mean(axis = 0)
        elif i>=(ttFr-win/2):
            ims_avg_ = ims[int(i-win/2):].mean(axis = 0)
        else:
            ims_avg_ = ims[int(i-win/2):int(i+win/2)].mean(axis = 0)
        diffs.append(cv2.absdiff(ims_avg_.astype('uint8'), ims[i].astype('uint8')))
    diffs = np.array(diffs)
    ims_avg = ims.mean(axis = 0)
    return diffs, ims_avg

def delta_frame(ims: np.ndarray, win = 500):
    ims = ims.astype('uint8')
    ttFr = ims.shape[0]
    diffs =[]
    for i in range(ttFr):
        if i<=win/2:
            ims_avg_ = ims[0:int(i+win/2)].mean(axis = 0)
        elif i>=(ttFr-win/2):
            ims_avg_ = ims[int(i-win/2):].mean(axis = 0)
        else:
            ims_avg_ = ims[int(i-win/2):int(i+win/2)].mean(axis = 0)
        diffs.append(cv2.absdiff(ims_avg_.astype('uint8'), ims[i].astype('uint8')))
    diffs = np.array(diffs)
    ims_avg = ims.mean(axis = 0)
    return diffs, ims_avg

def delta_frame(ims: np.ndarray):
    ims = ims.astype('float32')
    ttFr = ims.shape[0]
    ims_avg = ims.mean(axis = 0)
    dF = np.abs(ims_avg[None,:,:].astype('float32') -ims.astype('float32'))
    return dF.astype('uint8'), ims_avg.astype('uint8')


def convert2array(file, fx = 1, ds = 1,blur = 3,  opt = 'GRAY'):
    length, width, height, fps = get_stats(file)
    ims = []
    cap = cv2.VideoCapture(file)
    if opt=='GRAY':
        conv = cv2.COLOR_BGR2GRAY
    else:
        conv = cv2.COLOR_BGR2HSV
    for i in range(length):
        if i % ds == 0:
            frame = grab_frame(cap, fx = fx)
            frame = cv2.cvtColor(frame, conv)
            ims.append(cv2.medianBlur(frame,blur))
#             ims.append(frame)
#             ims.append(cv2.GaussianBlur(frame, (3, 3), 1))
    return np.array(ims)

def erode(im, ker = 5, ite = 1):
    kernel = np.ones((ker,ker), np.uint8)
    ims = []
    for i in range(im.shape[0]):
        ims.append(cv2.erode(im[i], kernel, iterations=ite))
    return np.array(ims).astype('uint8') 

def dilate(im, ker = 5, ite = 1):
    kernel = np.ones((ker,ker), np.uint8)
    ims = []
    for i in range(im.shape[0]):
        ims.append(cv2.dilate(im[i], kernel, iterations=ite))
    return np.array(ims).astype('uint8') 

def plot_pts(masked, im, T):
    fr_num = masked.shape[0]
    im_ = np.repeat(im.copy()[:,:,:,None], repeats = 3,axis = 3).astype('uint8')
#     im_ = np.copy(masked)
    preFr = 50
    cols = np.random.randint(0, 255,(T.shape[0],3))
    for fr in range(fr_num):
        for f in range(T.shape[0]):
            c = cols[f]
            if int(T[f,fr,0])>0:
                im_[fr] = cv2.circle(im_[fr], (int(T[f,fr,0]), int(T[f,fr,1])), radius=1, color=(int(c[0]), int(c[1]), int(c[2])), thickness=3)
            if fr>25:
                for pre_fr in range(preFr):
                    if int(T[f,fr-pre_fr,0])>0:
                        if int(T[f,fr-pre_fr,1])>0:
                            im_[fr] = cv2.circle(im_[fr], (int(T[f,fr-pre_fr,0]), int(T[f,fr-pre_fr,1])), radius=1, \
                                         color=(int(c[0]), int(c[1]), int(c[2])), thickness=1)
                        
    return im_ 

### get basic stats, blur and cover to grey scale

In [11]:
length, width, height, fps = get_stats(file)
print('length: {0}, \nwidth: {1}, \nheight: {2}, \nfps:{3}'.format(\
            length, width, height, fps))
imsHSV = convert2array(file, fx = 0.5, ds = 2, opt = 'HSV')
ims = convert2array(file, fx = 0.5, ds = 2, opt = 'GRAY')
print(ims.shape)

length: 1021, 
width: 640, 
height: 480, 
fps:8.0
(511, 240, 320)


## view raw and selct upper and lower bounds

In [7]:
from PyQt5 import QtGui 

In [12]:
dF, ims_avg = delta_frame(ims)

In [13]:
tmp = dF[0]
tmp.shape

(240, 320)

In [15]:
imv = RawImageWidget(scaled=True)

In [17]:
imv.setImage(foo) 

In [2]:
import cv2

In [3]:
camera = cv2.VideoCapture(3) 

In [ ]:
hue = imsHSV[:,:,:,0]
saturation = imsHSV[:,:,:,1]
value = imsHSV[:,:,:,2]

In [ ]:
tmp = (hue>20)*(hue<100)*1

In [2]:
pwd

'/Users/virginiarutten/Documents/deep_fish/tracking'

In [ ]:
foo = np.zeros_like(hue)
foo[tmp]= hue[tmp]

In [ ]:
pg.image

In [ ]:
pg.image(tmp)

In [ ]:
image(ims, title='raw')

In [ ]:
s = image(saturation, title='saturation')
v = image(value, title='value')

In [ ]:
dFS, ims_avgS = delta_frame(saturation)
dFV, ims_avgV = delta_frame(value)

In [ ]:
image(dFS, title='dFsaturation')

In [ ]:
mask = dFS>26
mask = mask*1

In [ ]:
win = image(255-dFD, title='dF max area')
win2 = image(255-dFD, title='dF min area')
print(win2.roi.size()[0]*win2.roi.size()[1])
print(win.roi.size()[0]*win.roi.size()[1])

In [ ]:
image(mask, title='dFmask')

In [ ]:
image(dFV, title='dFvalue')

In [ ]:
dFE = erode(mask.astype('uint8'), ker = 2, ite = 1)
image(dFE, title='dFE')


In [ ]:
dFD = dilate(dFE, ker = 3, ite = 1)
image(dFD, title='dFD')

In [ ]:
im_, keypoints = countour_frames(dFD*255,area_mn = 45, area_mx = 400, fish_num = 3)

In [ ]:
win = image(im_)

## find trajectories

In [ ]:
mask.shape

In [ ]:
im_.shape

In [ ]:
min_dist = 30
trajs = find_trajs(keypoints, min_dist)

In [ ]:
pl.matshow(trajs[:,:,0])

In [ ]:
imT = plot_pts(mask,ims, trajs)

In [ ]:
imM = plot_pts(mask,mask, trajs)

In [ ]:
win = image(imM, 'imM')

In [ ]:
win = image(imT)

In [ ]:
vPath = np.sum(trajs[:,:,0]>0, axis = 1)
T = trajs[vPath>10,:,:]

In [ ]:
imT.shape

In [ ]:
win = image(imT)
win.play(rate = 30)

In [ ]:
MyGif('test_hsv', imT,fps = 30)

In [ ]:
for i in range(trajs.shape[0]):
    tmp = trajs[i,:,:][trajs[i,:,0]>0,:]
    pl.plot(tmp[:,0],tmp[:,1], 'o')

In [ ]:
T = trajs

In [ ]:
title = 'trajectories'
win = pg.PlotItem(title=title)
win.showGrid(x=True, y=True)
# win.addlegend()
for i in range(T.shape[0]):
    tmp = T[i,:,:][T[i,:,0]>0,:]
    win.plot(tmp[:,0],tmp[:,1], pen=(i,3), symbol = 'o')
    

In [ ]:
xx = []
for i in range(T.shape[0]):
    inds = T[i,:,0]>0
    xx.append(T[i,inds,:])
xx = np.array(xx)

In [ ]:
X = np.array([[(np.linalg.norm(T[fish][ind+1,:]-pre)) for ind, pre in enumerate(T[fish][:-1,:])] for fish in range(T.shape[0]) ])

In [ ]:
X.shape

In [ ]:
X[X>min_dist] = 0

In [ ]:

X = np.array([[(np.linalg.norm(xx[fish][ind+1,:]-pre)) for ind, pre in enumerate(xx[fish][:-1,:])] for fish in range(len(xx)) ])

In [ ]:
pg.plot(np.mean(X, axis = 0))

In [ ]:
pg.plot??

In [ ]:
title = 'swim distribution3'
win = pg.plot(title=title)
win.plotItem.addLegend()
win.plotItem.titleLabel.setText(title)
win.win.move(0,0)
win.showGrid(x=True, y=True)
win.plot(np.mean(X, axis = 0), name='distance')


In [ ]:
for i in range(X.shape[0]):
    pl.plot(X[i])

In [ ]:
X.shape

In [ ]:
title = 'trajectories'
win = pg.plot(title=title)
win.showGrid(x=True, y=True)
# win.addlegend()
for i in range(10):
    tmp = T[i,:,:][T[i,:,0]>0,:]
    win.plot(tmp[:,0],tmp[:,1], pen=(i,3), symbol = 'o')
    

In [ ]:
params = cv2.SimpleBlobDetector_Params()
params.filterByArea = True
params.minArea = 20
params.maxArea = 10000
params.filterByInertia = False
params.maxInertiaRatio = 1
params.minThreshold = 0
params.maxThreshold = 256

detector = cv2.SimpleBlobDetector_create(params)

kpts = []
tmp_ims = []
im_avg_kp = np.copy(ims_avg)

for i in range(dFD.shape[0]-60):
    tmp_im = 255-dFD[i]*255
    keypoints = detector.detect(tmp_im)
    kpts.append(kpts)
    im_with_keypoints = cv2.drawKeypoints(tmp_im, keypoints, np.array([]), (255,0,255), cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)
    im_with_keypoints = cv2.drawKeypoints(tmp_im, keypoints, np.array([]), (255,0,255), cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)
    im_avg_kp = cv2.drawKeypoints(im_avg_kp, keypoints, np.array([]), (255,0,255), cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)
    
    tmp_ims.append(im_with_keypoints) 
    
tmp_ims = np.array(tmp_ims)
# kpts
win = image(im_avg_kp)
win1 = image(tmp_ims)
win1.move(500,0)

In [ ]:
# lLim = int(np.percentile(fD.flatten(), 97.0))
# uLim = 255


In [ ]:
pg.image(im_avg_kp)

In [ ]:
pg.image(tmp_ims)

In [ ]:
win2 = pg.image(tmp_ims, 'tmp_ims')
win2.move(0,0)

## simple pyqtDemo

In [ ]:
file_path = '/Users/virginiarutten/Documents/deep_fish/assets/pavlov_dog.jpg'
pp = np.array(Image.open(file_path))
ss = pp.shape
print(ss)

In [ ]:
win = QtGui.QWidget()
title='you can do it!'
win.resize(1200,800)
win.setWindowTitle(title)
win.move(0,0)

##create qWidget
imv1= RawImageWidget(scaled=True)

# imv2= pg.ImageView()
# l2 = imv2.layout()
layout = QtGui.QGridLayout()
win.setLayout(layout)

layout.addWidget(imv1)
# layout.addWidget(imv2)
imv1.setImage(pp.T)
# imv2.setImage(pp.T, scale = (3,3), pos= (0,0), levels = (10,90),\
#              autoHistogramRange= False, autoRange = True)
win.show()


In [ ]:
filename1 = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")

In [ ]:
A = MyArduino(port='/dev/cu.usbmodem14511')

In [ ]:
A.sendChar('1')

In [ ]:
path = '/Users/virginiarutten/Documents/deep_fish/tracking'
os.chdir(path)
os.getcwd()
file = glob('../videos/*')[1]
print(file)

## save new video

In [ ]:
cap = cv2.VideoCapture(1)
frame, imW, imH = grab_frame(cap, fx=.5)
print(imW, imH)

In [ ]:
def check_file(path, file):
    i = 0
    if len(glob(path+file))>0:
        i +=1
        print(i)
        file = str(i)+file
    else:
        print('file is fine')
    return file

In [ ]:
path = '/Users/virginiarutten/Documents/deep_fish/videos/'
file = 'test0.avi'

In [ ]:
file = check_file(path, file)
print(file)

## basic detection

In [ ]:
fD_rgb, fD, keypoints = detect_fish(frameDelta, ims_rgb, p_val, area_mn)

In [ ]:
len(keypoints)

In [ ]:
pg.image(fD)

In [ ]:
trajs[0:4,0,:] 

In [ ]:
fD_traj.shape

In [ ]:
mv = pg.image(fD_traj)

In [ ]:
mv.play(25)

In [ ]:
d_fps

In [ ]:
clip = gif('180331_test_tracking_sc2_fps2', fD_traj,fps=d_fps*2, scale=2.0)

In [ ]:
mv = pg.image(fD_traj)

In [ ]:
mv.play(rate = 25)

In [ ]:
tmp = np.repeat(frameDelta.copy()[:,:,:,None], repeats = 3,axis = 3)

In [ ]:
tmp.shape

In [ ]:
ims_avg.shape

In [ ]:
cv2.circle

In [ ]:
pl.plot(trajs[0][:,0], trajs[0][:,1])

In [ ]:
pg.image(fD_traj)

In [ ]:
trajs[0][0][0]

In [ ]:
sg = pg.plot(trajs[0][:,0],trajs[0][:,1])

In [ ]:
sg??

In [ ]:
sg = pg.ScatterPlotItem(pos = trajs[0])

In [ ]:
loc


In [ ]:
np.linalg.norm(loc-pre_loc)

In [ ]:
ord_

In [ ]:
i

In [ ]:
ord_

In [ ]:
locs[2,0]

In [ ]:
np.linalg.norm(tmp[0]-tmp[1])

In [ ]:
tmp[0]

In [ ]:
tmp = keypoints[0].T

In [ ]:
keypoints[]

In [ ]:
tmp = fD_rgb.copy()
tmp[:,:,:,0] = tmp[:,:,:,2].copy()
tmp[:,:,:,2] = fD_rgb.copy()[:,:,:,0]

In [ ]:
pg.image(tmp)
pg.image(fD)

In [ ]:
params = cv2.SimpleBlobDetector_Params() 
params.minThreshold = 200
params.maxThreshold = 265
params.filterByArea = True
params.minArea = 0
params.maxArea = 100*5000
params.filterByColor = True
params.blobColor = 255
detector = cv2.SimpleBlobDetector_create(params)
kk = detector.detect(mask[1].astype('uint8'))
kk

In [ ]:
tmp = cv2.drawKeypoints(ims[1,:,:], kk, np.array([]), \
                                                   (255,0,0),flags = 0)
for j in range(0,len(kk)):
    x_ = np.array([kk[j].pt[0]])
    y_ = np.array([kk[j].pt[1]])
    x = np.concatenate([x,x_])
    y = np.concatenate([y,y_])
    tmp = cv2.circle(tmp,\
                    (x_, y_), radius = int(kk[j].size), color=(255),\
                                               thickness=3)
    
    mask[1] = cv2.circle(mask[1],\
                    (x_, y_), radius = int(kk[j].size), color=(255),\
                                               thickness=3)    
pg.image(tmp)
pg.image(mask[1].astype('uint8'))

In [ ]:
ss = frameDelta.shape

In [ ]:
frameDelta.shape

In [ ]:
min_area = 300*100

In [ ]:
mask = cv2.inRange(fD,  p_val+10, 255)


In [ ]:
tmp = pg.image(fD[0], title='fD')

In [ ]:
roi = tmp.getRoiPlot()

In [ ]:
roi.setEnabled=True

In [ ]:
tmp.win.setWindowTitle('test')

In [ ]:
kk = tmp.roi.size()
tmp.roi.size()

In [ ]:
area_mn = kk.x()*kk.y()
print(area_mn)

In [ ]:
area_mx = kk.x()*kk.y()

In [ ]:
area_mx

In [ ]:
pg.image(fD)

In [ ]:
win = pg.image(fD[3])

In [ ]:
ROI = win.getRoiPlot()

In [ ]:
roi = ROI.getPlotItem()

In [ ]:
roi.size()

In [ ]:
%%time
fD = frameDelta.copy()
im_with_keypoints = np.zeros([ss[0],ss[1],ss[2],3])
keypoints = {}
# for i in range(0,round(ss[0])):

sizes = []
for i in range(30):
    if i % 25 ==0:
        print(i)
    _, cnts, _ = cv2.findContours(image = mask[i].copy(), mode = cv2.RETR_EXTERNAL,
            method = cv2.CHAIN_APPROX_SIMPLE)
        # loop over the contours
    for c in cnts:
        
        # if the contour is too small, ignore it
        
        if cv2.contourArea(c) < min_area:
            pass
        (x, y, w, h) = cv2.boundingRect(c)
        fD[i] = cv2.rectangle(fD[i], (x, y), (x + w, y + h), (0, 255, 0), 2)
#     kk = detector.detect(mask[i].astype('uint8'))
#     im_with_keypoints[i,:,:,:] = cv2.drawKeypoints(ims[i,:,:], kk, np.array([]), \
#                                                    (255,0,0),flags = 0)
#     keypoints[i] = np.array([[[],[]]])
#     x = np.array([])
#     y = np.array([])
#     for j in range(0,len(kk)):
#         x_ = np.array([kk[j].pt[0]])
#         y_ = np.array([kk[j].pt[1]])
#         x = np.concatenate([x,x_])
#         y = np.concatenate([y,y_])
#         im_with_keypoints[i,:,:,:] = cv2.circle(im_with_keypoints[i,:,:,:],\
#                     (x_, y_), radius = int(kk[j].size), color=(255),\
#                                                thickness=3)
#         fD[i,:,:]  = cv2.circle(fD[i,:,:] ,\
#                     (x_, y_), radius = int(kk[j].size), color=(255),\
#                                                thickness=3)
# #         keypoints[i] = np.array([[kk[0].pt[0]],[kk[0].pt[0]]])
#     xx =x.reshape([1,-1])
#     yy =y.reshape([1,-1])
#     keypoints[i] = np.concatenate([xx,yy], axis = 0)   
    

In [ ]:
%%time
fD = frameDelta.copy()
im_with_keypoints = np.zeros([ss[0],ss[1],ss[2],3])
keypoints = {}
# for i in range(0,round(ss[0])):

for i in range(30):
    print(i)
    if i % 25 ==0:
        print(i)
    kk = detector.detect(mask[i].astype('uint8'))
    im_with_keypoints[i,:,:,:] = cv2.drawKeypoints(ims[i,:,:], kk, np.array([]), \
                                                   (255,0,0),flags = 0)
    keypoints[i] = np.array([[[],[]]])
    x = np.array([])
    y = np.array([])
    for j in range(0,len(kk)):
        x_ = np.array([kk[j].pt[0]])
        y_ = np.array([kk[j].pt[1]])
        x = np.concatenate([x,x_])
        y = np.concatenate([y,y_])
        im_with_keypoints[i,:,:,:] = cv2.circle(im_with_keypoints[i,:,:,:],\
                    (x_, y_), radius = int(kk[j].size), color=(255),\
                                               thickness=3)
        fD[i,:,:]  = cv2.circle(fD[i,:,:] ,\
                    (x_, y_), radius = int(kk[j].size), color=(255),\
                                               thickness=3)
#         keypoints[i] = np.array([[kk[0].pt[0]],[kk[0].pt[0]]])
    xx =x.reshape([1,-1])
    yy =y.reshape([1,-1])
    keypoints[i] = np.concatenate([xx,yy], axis = 0)   
 

In [ ]:
pg.image(255-mask.astype('uint8'))

In [ ]:
# mask = cv2.inRange(fD,  p_val+20, 255)
# pg.image(mask)
pg.image(fD)

In [ ]:
pg.image(mask[i].astype('uint8'))

In [ ]:
pg.image(fD)

In [ ]:
keypoints[3]

In [ ]:
pg.image(im_with_keypoints)

In [ ]:
pg.image(frameDelta[i,:,:].astype('uint8'))

In [ ]:
fr_num = frameDelta.shape[0]
thresh = cv2.threshold(frameDelta.reshape([fr_num, -1]), p_val, p_val+20, cv2.THRESH_BINARY)[1]
thresh = thresh.reshape([fr_num,height,width])

In [ ]:
pg.image(thresh)

In [ ]:
ret, frame = cap.read()

In [ ]:
gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
gray = cv2.GaussianBlur(gray, (3, 3), 1)

In [ ]:
pg.image(frame_delta)

In [ ]:
T.grab_frame()
T.process_frame()
T.get_frame_contours()
T.show_frame()

In [ ]:
T.recording = False

In [ ]:
T.camera.release()
cv2.destroyAllWindows() 

In [ ]:
T.kill_threads()

In [ ]:
path = './videos'

In [ ]:
alias test echo python here %s

In [ ]:
%who

In [ ]:
# camera = cv2.VideoCapture(0)
# time.sleep(0.25)
# firstFrame = None

In [ ]:
cv2.imshow('img',np.random.randn(100,100))
cv2.waitKey(0)
cv2.destroyAllWindows()


In [ ]:
# def main():
#     print('starting')
#     T = MyTracker()
#     print('Tracker created')
#     T.start_recording()
    
    
# if __name__ == '__main__':
#     print('hello')
#     main()

In [ ]:
T = MyTracker()

In [ ]:
T = MyTracker()

In [ ]:
T.grab_frame()

In [ ]:
T.kill_threads()

In [ ]:
camera.release()
cv2.destroyAllWindows()  

In [ ]:
T.start_recording()

In [ ]:
# loop over the frames of the video
ims = []

while True:
    sleep
    print("frame grabbed")
    # grab the current frame and initialize the occupied/unoccupied
    # text
    (grabbed, frame) = camera.read()
    text = "Unoccupied"

    # if the frame could not be grabbed, then we have reached the end
    # of the video
    if not grabbed:
        print('error')
        break

    # resize the frame, convert it to grayscale, and blur it
    frame = imutils.resize(frame, width=500)
    ims.append(frame)
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    gray = cv2.GaussianBlur(gray, (21, 21), 0)

    # if the first frame is None, initialize it
    if firstFrame is None:
        firstFrame = gray
        continue
        
    # compute the absolute difference between the current frame and
    # first frame
    frameDelta = cv2.absdiff(firstFrame, gray)
    thresh = cv2.threshold(frameDelta, 25, 255, cv2.THRESH_BINARY)[1]

    # dilate the thresholded image to fill in holes, then find contours
    # on thresholded image
    thresh = cv2.dilate(thresh, None, iterations=2)
    _, cnts, _ = cv2.findContours(image = thresh.copy(), mode = cv2.RETR_EXTERNAL,
        method = cv2.CHAIN_APPROX_SIMPLE)


    # loop over the contours
    for c in cnts:
        # if the contour is too small, ignore it
        if cv2.contourArea(c) < args["min_area"]:
#             print('nothing')
            continue

        # compute the bounding box for the contour, draw it on the frame,
        # and update the text
        (x, y, w, h) = cv2.boundingRect(c)
        cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)
        text = "Occupied"
        
    # draw the text and timestamp on the frame
    cv2.putText(frame, "Room Status: {}".format(text), (10, 20),
        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 2)
    cv2.putText(frame, datetime.datetime.now().strftime("%A %d %B %Y %I:%M:%S%p"),
        (10, frame.shape[0] - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.35, (0, 0, 255), 1)

    # show the frame and record if the user presses a key
    cv2.imshow("Security Feed", frame)
    cv2.imshow("Thresh", thresh)
    cv2.imshow("Frame Delta", frameDelta)
    key = cv2.waitKey(1) & 0xFF

    # if the `q` key is pressed, break from the lop
    if key == ord("q"):
        break    
        


In [ ]:
# cleanup the camera and close any open windows
camera.release()
cv2.destroyAllWindows()     

ims = np.array(ims)v

In [ ]:
cv2.contourArea(cc[0])

In [ ]:
pl.imshow(thresh)

In [ ]:
cv2.findContours??

In [ ]:
pg.image(ims)

In [ ]:
pl.imshow(firstFrame)